In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import requests, re, time
from bs4 import BeautifulSoup
import scraping_class
import pprint
import pandas as pd
logfile = 'log.csv'## name your log file.
connector = scraping_class.Connector(logfile)
session = requests.session()
session.headers['wmd776@alumni.ku.dk'] = 'wmd776@alumni.ku.dk' 
session.headers['Amalie Tokkesdal'] = 'Amalie Tokkesdal'
session.headers

{'User-Agent': 'python-requests/2.21.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'wmd776@alumni.ku.dk': 'wmd776@alumni.ku.dk', 'Amalie Tokkesdal': 'Amalie Tokkesdal'}

# Importing the packages

# Importing and parsing the data

In [2]:
url='https://www.imdb.com/chart/top'
number_of_movies = 250
response = requests.get(url)

if response.ok:
    html = response.text
else:
    print('error')
    
soup = BeautifulSoup(html,'html.parser')

link_location = html.split('href="')[1:]

links = set()


for link_loc in link_location:
    link = link_loc.split('"')[0]
    if '/title/' in link:
        links.add(link) 
links = ['https://www.imdb.com/'+link for link in links if not 'ref' in link]# add the domain to each link

links

['https://www.imdb.com//title/tt3011894/',
 'https://www.imdb.com//title/tt0113277/',
 'https://www.imdb.com//title/tt0892769/',
 'https://www.imdb.com//title/tt0167404/',
 'https://www.imdb.com//title/tt0097576/',
 'https://www.imdb.com//title/tt0094625/',
 'https://www.imdb.com//title/tt0088763/',
 'https://www.imdb.com//title/tt0047296/',
 'https://www.imdb.com//title/tt0047396/',
 'https://www.imdb.com//title/tt0071853/',
 'https://www.imdb.com//title/tt1895587/',
 'https://www.imdb.com//title/tt0347149/',
 'https://www.imdb.com//title/tt0019254/',
 'https://www.imdb.com//title/tt0169858/',
 'https://www.imdb.com//title/tt0057115/',
 'https://www.imdb.com//title/tt0045152/',
 'https://www.imdb.com//title/tt0054215/',
 'https://www.imdb.com//title/tt0105695/',
 'https://www.imdb.com//title/tt0118694/',
 'https://www.imdb.com//title/tt0120735/',
 'https://www.imdb.com//title/tt0978762/',
 'https://www.imdb.com//title/tt0092005/',
 'https://www.imdb.com//title/tt0087884/',
 'https://w

In [3]:
def find_budget(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Budget:":
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))
        
def find_gross(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Cumulative Worldwide Gross:":
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))
        
def find_grossUSA(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Gross USA:":
            return float(i.text.strip().split("\n")[0].split("$")[1].replace(",", ""))

def find_rating(soup):
    for i in soup.find(id="title-overview-widget").find_all("div", class_="ratingValue"):
        return i.text.strip().split("/")[0]
        
def find_runtime(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Runtime:":
            return float(i.text.strip().split("\n")[1].split("min")[0].strip())
def find_date(soup):
    for i in soup.find(id="titleDetails").find_all("div", class_="txt-block"):
        if i.find().text == "Release Date:":
            return i.text.strip().split(":")[1].split("(")[0].strip()
        
def find_genre(soup):
    for i in soup.find(id="title-overview-widget").find_all("div", class_="subtext"):
        if str(i.text.strip().split("|")[2].replace('\n','').replace(' ','').split(","))[-3]==(')'):
            return i.text.strip().split("|")[1].replace('\n','').replace(' ','').split(",")
        else:
            return i.text.strip().split("|")[2].replace('\n','').replace(' ','').split(",")
        
def preparedatetime(dataframe):
    dataframe['datetime'] = pd.to_datetime(dataframe['date'], format = '%d %B %Y')
    dataframe['month'] = dataframe.datetime.dt.month
    dataframe['month'] = dataframe.datetime.dt.day
    dataframe['month'] = dataframe.datetime.dt.year

In [8]:

def page_extractor(url):
    r, call_id = connector.get(url, "LOL")
    try:
        soup = BeautifulSoup(r.text, "html.parser")
        title = soup.select("#title-overview-widget > div.vital > div.title_block > div > div.titleBar > div.title_wrapper > h1")[0].text.split("\xa0")[0]
        budget = find_budget(soup)
        cumulative_worldwide_gross=find_gross(soup)
        gross_usa=find_grossUSA(soup)
        rating = find_rating(soup)
        runtime = find_runtime(soup)
        date= find_date(soup)
        genre = find_genre(soup)
    except:
        soup = None
        title = None
        budget = None
        cumulative_worldwide_gross= None
        gross_usa = None
        rating=None
        runtime = None
        date = None
        genre = None
    return {
        "title": title,
        #"year": year,
        "budget": budget,
        "url": url,
        "cumulative_world_gross": cumulative_worldwide_gross,
        "gross_usa": gross_usa,
        "rating": rating,
        "runtime": runtime,
        "date": date,
        "genre": genre
    }

#Husk at finde titel på engelsk

### Create a list and append information from every movie on the list. 
Remark: We have only chosen the first 15 so far, as to not make the coding too slow in progress. If you wish to see the whole list simply delete the [:15]


In [9]:
res = []
for idx, i in enumerate(links[:15]):
    print(idx + 1, "of", len(links[:15]))
    res.append(page_extractor(i))


1 of 15
2 of 15
3 of 15
4 of 15
5 of 15
6 of 15
7 of 15
8 of 15
9 of 15
10 of 15
11 of 15
12 of 15
13 of 15
14 of 15
15 of 15


### Making our list into a dataframe. 

A dataframe is basically just when a list has both several columns and rows, if there's only one row it's an array.

In [11]:
df = pd.DataFrame(res).dropna()

df = df[df.date != '1997']


#df['datetime'] = pd.to_datetime(df['date'], format = '%d %B %Y')
#df = pd.concat([df, pd.get_dummies(df['genre'], drop_first = True)], axis=1)
#var_list = list(range(1900,2020))
#df.query('variable in var_list')

df

def preparedatetime(dataframe):
    dataframe['datetime'] = pd.to_datetime(dataframe['date'], format = '%d %B %Y')
    dataframe['year'] = dataframe.datetime.dt.year
    dataframe['month'] = dataframe.datetime.dt.month
    dataframe['day'] = dataframe.datetime.dt.day
    
    
preparedatetime(df)

df

,budget,cumulative_world_gross,date,genre,gross_usa,rating,runtime,title,url,datetime,year,month,day
0,3300000.0,36353750.0,7 May 2015,"[Comedy, Drama, Thriller]",3107072.0,8.1,122.0,Wild Tales,https://www.imdb.com//title/tt3011894/,2015-05-07,2015,5,7
1,60000000.0,107000000.0,9 February 1996,"[Crime, Drama, Thriller]",67436818.0,8.2,170.0,Heat,https://www.imdb.com//title/tt0113277/,1996-02-09,1996,2,9
2,165000000.0,494878759.0,25 March 2010,"[Animation, Action, Adventure]",217581231.0,8.1,98.0,Sådan træner du din drage,https://www.imdb.com//title/tt0892769/,2010-03-25,2010,3,25
3,40000000.0,672806292.0,6 January 2000,"[Drama, Mystery, Thriller]",293506292.0,8.1,107.0,Den sjette sans,https://www.imdb.com//title/tt0167404/,2000-01-06,2000,1,6
4,48000000.0,474171806.0,15 September 1989,"[Action, Adventure]",197171806.0,8.2,127.0,Indiana Jones og det sidste korstog,https://www.imdb.com//title/tt0097576/,1989-09-15,1989,9,15
6,19000000.0,381109762.0,13 December 1985,"[Adventure, Comedy, Sci-Fi]",210609762.0,8.5,116.0,Tilbage til fremtiden,https://www.imdb.com//title/tt0088763/,1985-12-13,1985,12,13
8,1000000.0,36888916.0,25 April 1955,"[Mystery, Thriller]",36764313.0,8.5,112.0,Skjulte øjne,https://www.imdb.com//title/tt0047396/,1955-04-25,1955,4,25
10,20000000.0,98275238.0,25 February 2016,"[Biography, Crime, Drama]",45055776.0,8.1,129.0,Spotlight,https://www.imdb.com//title/tt1895587/,2016-02-25,2016,2,25
11,24000000.0,235184110.0,14 October 2005,"[Animation, Adventure, Family]",4711096.0,8.2,119.0,Det levende slot,https://www.imdb.com//title/tt0347149/,2005-10-14,2005,10,14


# Trying to make an OLS regression

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)
#(X_train - mean(X_train) / sd(X_train)

scaled_X_train.mean(axis=0)

In [ ]:
scaled_X_test.mean(axis=0)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error as mse

X = np.array(df[['soup',
        'title',
        'budget',
        'gross_usa',
        'rating',
        'runtime',
        'date',
        'genre']])
y = np.array(df['cumulative_worldwide_gross'])

def rmse(y_pred, y_true):
    return np.sqrt(mse(y_pred, y_true))

output=[]

for random_state in range(10):
    X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=random_state)
    X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=1/2, random_state=random_state)
    
    reg = LinearRegression().fit(X_train, y_train)
    reg.predict(X_test)
    
    output.append([rmse(reg.predict(X_val), y_val),
                  rmse(reg.predict(X_test), y_test)])
    
print(pd.DataFrame(output, columns=['test', 'validation']))

print(pd.DataFrame(output, columns=['test', 'validation']).describe())

## Machine learning

From computing the descriptive statistics of RMSE for the train data on  the test and validation data we can conclude we can split the train data and use this to fit the model. This is because the descriptive statistics are quite similar and we therefore presume our model i generalizing enough. (Balanced). 



In [ ]:
url_ = links[0]
r = requests.get(url_)

url_

In [ ]:
soup_ = BeautifulSoup(r.text,'html.parser')

In [ ]:


def getdummies(i):
    pd.get_dummies(genre['sex'], drop_first = True)



df['drama'] = pd.get_dummies(genre['sex'], drop_first = True)

df['smokerd'] = pd.get_dummies(tips['smoker'], drop_first = True)

tips['timed'] = pd.get_dummies(tips['time'], drop_first = True)

tips = pd.concat([tips, pd.get_dummies(tips['day'], drop_first = True)], axis=1)

tips.head()
